In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd

# Load dataset (make sure the file is in the same folder as this script or give full path)
df = pd.read_csv("job_dataset.csv")

# Preview first 5 rows
print("🔹 First 5 Rows:")
print(df.head(), "\n")

# Dataset shape (rows, columns)
print("🔹 Dataset Shape:", df.shape, "\n")

# Column names
print("🔹 Columns in Dataset:")
print(df.columns.tolist(), "\n")

# Basic info about columns (dtype, non-null count)
print("🔹 Dataset Info:")
print(df.info(), "\n")

# Summary statistics (for numeric columns)
print("🔹 Summary Statistics:")
print(df.describe(), "\n")

# Count missing values in each column
print("🔹 Missing Values per Column:")
print(df.isnull().sum(), "\n")


Saving job_dataset.csv to job_dataset.csv
🔹 First 5 Rows:
    job_id              job_title  salary_usd salary_currency  \
0  AI00001  AI Research Scientist       90376             USD   
1  AI00002   AI Software Engineer       61895             USD   
2  AI00003          AI Specialist      152626             USD   
3  AI00004           NLP Engineer       80215             USD   
4  AI00005          AI Consultant       54624             EUR   

  experience_level employment_type company_location company_size  \
0               SE              CT            China            M   
1               EN              CT           Canada            M   
2               MI              FL      Switzerland            L   
3               SE              FL            India            M   
4               EN              PT           France            S   

  employee_residence  remote_ratio  \
0              China            50   
1            Ireland           100   
2        South Korea        

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("job_dataset.csv")

# --- 1. Drop unneeded columns ---
# 'job_id', 'posting_date', 'application_deadline' may not be useful for ML
df.drop(columns=["job_id", "posting_date", "application_deadline"], inplace=True)

# --- 2. Standardize text columns ---
text_cols = ["job_title", "company_location", "employee_residence",
             "experience_level", "employment_type", "company_size",
             "industry", "company_name", "education_required"]

for col in text_cols:
    df[col] = df[col].str.strip().str.lower()

# --- 3. Clean skills list ---
# Split comma-separated skills, strip spaces, lowercase
df["required_skills"] = df["required_skills"].apply(
    lambda x: [s.strip().lower() for s in x.split(",")] if isinstance(x, str) else []
)

# --- 4. Salary preprocessing ---
# Already numeric (salary_usd), but let's create categories for dashboards
df["salary_category"] = pd.qcut(df["salary_usd"], q=4,
                                labels=["low", "mid", "high", "very high"])

# --- 5. Experience buckets ---
df["experience_bucket"] = pd.cut(df["years_experience"],
                                 bins=[0,2,5,10,20],
                                 labels=["junior","mid","senior","expert"])

# --- 6. Remote work classification ---
df["remote_type"] = df["remote_ratio"].apply(
    lambda x: "onsite" if x==0 else ("remote" if x==100 else "hybrid")
)

# --- 7. Benefits score category ---
df["benefits_category"] = pd.cut(df["benefits_score"],
                                 bins=[0,6,8,10],
                                 labels=["low","medium","high"])

# --- 8. Save cleaned dataset ---
df.to_csv("job_dataset.csv", index=False)

print("✅ Dataset cleaned and saved as AI_Jobs_Cleaned.csv")
print(df.head(5))


✅ Dataset cleaned and saved as AI_Jobs_Cleaned.csv
               job_title  salary_usd salary_currency experience_level  \
0  ai research scientist       90376             USD               se   
1   ai software engineer       61895             USD               en   
2          ai specialist      152626             USD               mi   
3           nlp engineer       80215             USD               se   
4          ai consultant       54624             EUR               en   

  employment_type company_location company_size employee_residence  \
0              ct            china            m              china   
1              ct           canada            m            ireland   
2              fl      switzerland            l        south korea   
3              fl            india            m              india   
4              pt           france            s          singapore   

   remote_ratio                                    required_skills  \
0            50    

In [ ]:
import pandas as pd
import sqlite3

# Load cleaned data
df = pd.read_csv("job_dataset.csv")

# Save to SQLite (data warehouse style)
conn = sqlite3.connect("skill360_jobs.db")
df.to_sql("jobs", conn, if_exists="replace", index=False)

print("✅ ETL Pipeline Complete: Data stored in SQLite DB & CSV")


✅ ETL Pipeline Complete: Data stored in SQLite DB & CSV


In [ ]:
from google.colab import files
files.download("job_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>